In [1]:
import numpy as np
import math

In [52]:
data = np.load('./data/Position_task_with_dots_synchronised_min.npz')
X = data['EEG']
y = data['labels']

In [73]:
scoring2 = (lambda y, y_pred: np.linalg.norm(y - y_pred, axis=0).mean()) # Euclidean distance

In [54]:
cnn_pred = np.load('./runs/1631608603_Position_task_dots_min_pytorch_patience_10_lr_1e-05/checkpoint/run1/CNN_pred.npz')

In [55]:
pred = cnn_pred['pred']

In [56]:
import math 

def split(ids, train, val, test):
    assert (train+val+test == 1)
    IDs = np.unique(ids)
    num_ids = len(IDs)

    # priority given to the test/val sets
    test_split = math.ceil(test * num_ids)
    val_split = math.ceil(val * num_ids)
    train_split = num_ids - val_split - test_split

    train = np.isin(ids, IDs[:train_split])
    val = np.isin(ids, IDs[train_split:train_split+val_split])
    test = np.isin(ids, IDs[train_split+val_split:])

    return train, val, test

In [57]:
trainX = X 
trainY = y
ids = trainY[:, 0]

In [58]:
train, val, test = split(ids, 0.7, 0.15, 0.15)
X_train, y_train = trainX[train], trainY[train]
X_val, y_val = trainX[val], trainY[val]
X_test, y_test = trainX[test], trainY[test]

In [59]:
labels = y_test[:, 1:]
labels

array([[392.7, 305.5],
       [681.3, 508.6],
       [389.5, 143.7],
       ...,
       [115.5, 306.1],
       [732. , 310.3],
       [632.2, 353.6]])

In [60]:
pred

array([[398.8539 , 223.47581],
       [534.44116, 303.25864],
       [293.83344, 353.68942],
       ...,
       [109.53944, 289.58093],
       [648.24115, 208.97931],
       [337.35126, 266.7133 ]], dtype=float32)

In [61]:
len(labels)

5201

In [71]:
len(pred)

5201

In [69]:
scoring2(labels, pred)

152.53453823871163

In [63]:
scoring2(labels[0], pred[0][0])

93.55652616981153

In [74]:
# screen is 800 x 600

CENTER_X = 400
CENTER_Y = 300 

bins = [[] for i in range(4)]
strings = ["bin 0: quadrant 1", 
"bin 1: quadrant 2",
"bin 2: quadrant 3",
"bin 3: quadrant 4"]
for i in range(len(labels)):
	x,y = labels[i]
	if x >= CENTER_X and y >= CENTER_Y:
		bins[0].append(scoring2(np.array([x,y]), pred[i]))
	elif x < CENTER_X and y >= CENTER_Y:
		bins[1].append(scoring2(np.array([x,y]), pred[i]))
	elif x < CENTER_X and y < CENTER_Y:
		bins[2].append(scoring2(np.array([x,y]), pred[i]))
	elif x >= CENTER_X and y < CENTER_Y:
		bins[3].append(scoring2(np.array([x,y]), pred[i]))

In [75]:
sum([len(list) for list in bins])

5201

In [76]:
for list, string in zip(bins, strings):
	print(f"{string}: mean euclidian dist {np.average(np.array(list)):.3f}, std: {np.std(np.array(list)):2f}, on {len(list)} samples")

bin 0: quadrant 1: mean euclidian dist 144.805, std: 86.515833, on 1560 samples
bin 1: quadrant 2: mean euclidian dist 176.678, std: 102.653861, on 1232 samples
bin 2: quadrant 3: mean euclidian dist 138.569, std: 94.247046, on 1092 samples
bin 3: quadrant 4: mean euclidian dist 150.684, std: 84.611185, on 1317 samples
